In [7]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [8]:
# import the package
import osmosispy
from osmosispy import utils

In [9]:
# configure
mnemonic_key = "fat patch excite gold bubble large tunnel vote fine title hover junior advice cable ordinary column mass aunt trigger lucky hope animal abandon mansion"

## Prepare the account

to start localnet clone the [osmosis-labs/osmosis](https://github.com/osmosis-labs/osmosis) repo and run the following command

```bash
make localnet-build
make localnet-start
```

In [10]:
# authorize in the localnet
network = osmosispy.Network.localnet()
trader = osmosispy.Sdk.authorize(key=mnemonic_key).with_network(network)

# check the address
print(trader.address)

osmo1jggt8pcj2d8m9n62luytf8sdncj5uxfs3su2my


## Check the balance of the account

In [11]:
from osmosis_proto.proto.cosmos.bank.v1beta1 import query_pb2 as bank_query
from osmosis_proto.proto.cosmos.bank.v1beta1 import query_pb2_grpc as bank_query_grpc

# check the balance

stubBank = bank_query_grpc.QueryStub(trader.query.chain_channel)

result = utils.deserialize(
    stubBank.AllBalances(
        bank_query.QueryAllBalancesRequest(address=trader.address)
    )
)

print(result["balances"])


[]


## Get pools

In [12]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import query_pb2 as gamm_query
from osmosis_proto.proto.osmosis.gamm.v1beta1 import query_pb2_grpc as gamm_query_grpc

# get pools

stubGamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)

result = utils.deserialize(
    stubGamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

print(result['pools'])


[]
